In [ ]:
import pandas as pd
import pandas_utils as pu

In [ ]:
libraries = {
  "mce": r"C:\git\msn_library\data\library\mce_library_all_cleaned.tsv",
  "nih": r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv",
  "mce_nencka": r"C:\git\msn_library\data\iocb_libraries\Radim_mce_complete_cleaned.tsv",
  "iocb_peptide": r"C:\git\msn_library\data\iocb_libraries\iocb_peptide_library_cleaned.tsv"
}

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value).copy()
  df = df[df["monoisotopic_mass"] > 114]
  df = df.drop_duplicates(["inchikey"]).set_index(["inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)

In [ ]:
merged_df

In [ ]:
merged_df[merged_df["mce"] == True]

In [ ]:
for key in libraries.items():
  